In [2]:
import torch
from utils import get_embedding
import pandas as pd
from tqdm import tqdm
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv

In [13]:
model_name = "text2vec-base"

In [14]:
embedding = get_embedding(model_name=model_name)

In [15]:
t = "刷美德：英国80后为何越来越传统"

In [19]:
fn = "thucnews/thucnews.csv"
df = pd.read_csv(fn)
df.shape

(3163, 11)

In [20]:
title_embeddings = torch.zeros(df.shape[0], 768)
title_embeddings.shape

torch.Size([3163, 768])

In [23]:
titles = df['title']
with torch.no_grad():
    i = 0
    for title in tqdm(titles):
        t_em = torch.tensor(embedding.embed_query(title), dtype=float)
        title_embeddings[i, :] = t_em
        i = i + 1
    print(i)

100%|██████████| 3163/3163 [01:16<00:00, 41.51it/s]

3163


In [3]:
with open("thucnews/views/pkl/title_embeddings.pickle", "rb") as file:
    title_embeddings = pickle.load(file)

In [22]:
title_embeddings.shape

torch.Size([3163, 768])

In [23]:
score = cosine_similarity(title_embeddings, title_embeddings)

In [6]:
percentage = np.sum(score > 0.5 ) / (score.size)
percentage

0.017435733613312077

In [24]:
np.sum(score >= 0.5)

174437

In [25]:
score[score < 0.5] = 0

In [27]:
ct = np.sum(score > 0, axis=1)
for i in range(len(ct)):
    if ct[i] == 1:
        print("有等于零的")
    print(i+1, ct[i])

1 24
2 25
3 179
4 38
5 217
6 27
7 7
8 102
9 28
10 157
11 18
12 83
13 50
14 139
15 5
16 98
17 18
18 14
19 193
20 36
21 16
22 41
23 26
24 13
25 122
26 48
27 9
28 3
29 14
30 47
31 86
32 41
33 12
34 8
35 8
36 15
37 80
38 17
39 47
40 55
41 32
42 46
43 14
44 170
45 29
46 30
47 39
48 75
49 4
50 29
51 12
52 5
53 11
54 11
55 42
56 19
57 17
58 15
59 14
60 54
61 13
62 24
63 7
64 50
65 170
66 33
67 56
68 7
69 10
70 73
71 9
72 26
73 72
74 192
75 61
76 38
77 78
78 4
79 11
80 64
81 4
82 46
83 4
84 7
85 87
86 31
87 27
88 84
89 4
90 62
91 64
92 14
93 4
94 56
95 37
96 16
97 31
98 4
99 15
100 12
101 189
有等于零的
102 1
103 3
104 23
105 29
106 28
107 60
108 29
109 50
110 40
111 223
112 3
113 25
114 30
115 23
116 31
117 9
118 98
119 66
120 23
121 30
122 5
123 5
124 15
125 201
126 11
127 50
128 86
129 123
130 31
131 84
132 59
133 46
134 11
135 8
136 7
137 37
138 122
139 9
140 27
141 38
142 5
143 7
144 3
145 49
146 7
147 74
148 77
149 57
150 62
151 88
152 186
153 10
154 12
155 9
156 24
157 68
158 6
159 58
160 7


In [18]:
np.fill_diagonal(score, 1.0)

In [17]:
save_path = "thucnews/views/thucnews_title.csv"
# 打开CSV文件
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入表头
    writer.writerow(['Row', 'Column', 'Value'])

    # 遍历数组并写入CSV文件
    for i in tqdm(range(score.shape[0])):
        for j in range(score.shape[1]):
            if score[i][j] > 0:
                writer.writerow([i, j, score[i, j]])

100%|██████████| 3163/3163 [00:13<00:00, 230.61it/s]


In [19]:
df = pd.read_csv(save_path)
df.shape

(26967, 3)

In [20]:
df.columns

Index(['Row', 'Column', 'Value'], dtype='object')

In [21]:
df.to_csv(save_path, sep=" ", header=False, index=False)